In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("output.csv")
data.head()

,type,media_url,caption,likes,comments
0,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @stevewinterphoto | One of the lucky ...,256591,1271
1,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Charlie Hamilton James @chamiltonjame...,587042,1692
2,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @babaktafreshi | A bird of freedom fl...,312169,3039
3,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Sara Hylton @sarahyltonphoto | Women ...,153079,394
4,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Ian Teh @iantehphotography | Palm oil...,163279,618


## Normalize Likes & Comments

In [42]:
Likes = data['likes']
Norm_Likes = (Likes-Likes.min())/(Likes.max()-Likes.min())
print(Norm_Likes.head())
print("The range of Norm_Likes is between %i and %i"%(Norm_Likes.max(),Norm_Likes.min()))
print()
Comments = data['comments']
Norm_Comments = (Comments-Comments.min())/(Comments.max()-Comments.min())
print(Norm_Comments.head())
print("The range of Norm_Comments is between %i and %i"%(Norm_Comments.max(),Norm_Comments.min()))

0    0.243893
1    0.996790
2    0.370521
3    0.008052
4    0.031291
Name: likes, dtype: float64
The range of Norm_Likes is between 1 and 0

0    0.350037
1    0.504023
2    0.996708
3    0.029261
4    0.111192
Name: comments, dtype: float64
The range of Norm_Comments is between 1 and 0


## Engagement Score

In [33]:
Engagement_Score = 0.4*Norm_Likes + 0.6*Norm_Comments
print(Engagement_Score.head())
Engagement_Median = Engagement_Score.median()
print("The Engagement_Median is %f"% Engagement_Median)

0    0.307579
1    0.701130
2    0.746233
3    0.020777
4    0.079232
dtype: float64
The Engagement_Median is 0.236224


## Determine Engagement [ 1 or 0 ] base on Engagement Score

In [35]:
Engagement = Engagement_Score.apply(lambda x: 1 if x >= Engagement_Median else 0)
print(Engagement.head())

0    1
1    1
2    1
3    0
4    0
dtype: int64


In [36]:
data['Norm_Likes'] = Norm_Likes
data['Norm_Comments'] = Norm_Comments
data['Engagement_Score'] = Engagement_Score
data['Engagement'] = Engagement

In [38]:
data.to_csv('TaskA.csv', index=False)